In [69]:
#ALL IMPORTS
import numpy as np
import pandas as pd

In [53]:
# READING DATA
season17_18 = pd.read_csv("17_18.csv")
season18_19 = pd.read_csv("18_19.csv")
season19_20 = pd.read_csv("19_20.csv")

In [74]:
class Football:
    
    def __init__(self, data, name):
        self.data=data
        self.cleanData()
        self.name=name
        self.teams = set(self.data.HomeTeam) | set(self.data.AwayTeam)
#         self.teams = {i for i in self.data.HomeTeam}
#         self.teams.update({i for i in self.data.AwayTeam})
    
    def getTeamWins(self):
        winTeams = dict()
        for i in self.teams:
            winTeams[i]=0
        
        for i in range(len(self.data)):
            if self.data.FTR[i] == 'A': #Away team won
                winTeams[self.data.AwayTeam[i]]+=1
            elif self.data.FTR[i] == 'H': #Home team won
                winTeams[self.data.HomeTeam[i]]+=1
        
        return winTeams
    
    def getTeamLoses(self):
        loseTeams = dict()
        for i in self.teams:
            loseTeams[i]=0
        
        for i in range(len(self.data)):
            if self.data.FTR[i] == 'A': #Away team won
                loseTeams[self.data.HomeTeam[i]]+=1
            elif self.data.FTR[i] == 'H': #Home team won
                loseTeams[self.data.AwayTeam[i]]+=1
        
        return loseTeams
        
        return None
    
    def getTeamDraws(self):
        drawTeams = dict()
        for i in self.teams:
            drawTeams[i] = 0
            
        for i in range(len(self.data)):
            if self.data.FTR[i] == 'D':
                drawTeams[self.data.HomeTeam[i]] += 1
                drawTeams[self.data.AwayTeam[i]] += 1
        
        return drawTeams
    
    def getTeamGames(self): #How many games were played by each team
        gamesCount = dict()
        for i in self.teams:
            gamesCount[i] = 0
        for i in range(len(self.data)):
            gamesCount[self.data.HomeTeam[i]]+=1
            gamesCount[self.data.AwayTeam[i]]+=1
        return gamesCount
    
    def getTeamPoints(self):
        points = dict()
        #WIN 3 POINTS
        #DRAW 1 POINT
        for i in self.teams:
            points[i] = 0
        for i in self.getTeamWins():
            points[i]=self.getTeamWins()[i]*3
        for i in self.getTeamDraws():
            points[i]+=self.getTeamDraws()[i]*1
        return points
        
    
    def getTeamsTotalGoals(self):
        teamTotalGoals = dict()
        for i in self.teams:
            teamTotalGoals[i] = 0
            
        for i in range(len(self.data)):
            teamTotalGoals[self.data.HomeTeam[i]] += self.data.FTHG[i]
            teamTotalGoals[self.data.AwayTeam[i]] += self.data.FTAG[i]
            
        return teamTotalGoals
    
    def cleanData(self):
        colums = ["HomeTeam", "AwayTeam", "FTHG", "FTAG", "FTR", "HTHG", "HTAG", "HTR", "Referee", "HS", "AS", "HST", "AST",
                 "HF", "AF", "HC", "AC", "HY", "AY", "HR", "AR", "B365H", "B365D", "B365A"]
        for col in self.data.columns:
            contains = False
            for j in colums:
                if col == j:
                    contains = True
            if not contains:
                self.data = self.data.drop(col, axis=1)
    
    def getWrongPrediction(self):
        #TODO find matches were odds and match results were not equal
        luckyLoser = dict()
        for i in self.teams:
            luckyLoser[i] = 0
        kokku=0
        for i in range(len(self.data)):
            homeOdd = float(self.data.B365H[i])
            drawOdd = float(self.data.B365D[i])
            awayOdd = float(self.data.B365A[i])
            
            if homeOdd == awayOdd: #If odds are same, then there is no right prediction
                continue
            if homeOdd < awayOdd and self.data.FTR[i]!='H' and self.data.FTR[i]!='D':
                #print("Home:",homeOdd,"Draw:",drawOdd,"Away:",awayOdd,self.data.HomeTeam[i],"VS",self.data.AwayTeam[i],"WON SECOND TEAM")
                luckyLoser[self.data.AwayTeam[i]]+=1
                kokku+=1
            elif homeOdd > awayOdd and self.data.FTR[i]!='A' and self.data.FTR[i]!='D':
                #print("Home:",homeOdd,"Draw:",drawOdd,"Away:",awayOdd,self.data.HomeTeam[i],"VS",self.data.AwayTeam[i],"WON FIRST TEAM")
                luckyLoser[self.data.HomeTeam[i]]+=1
                kokku+=1
        print(self.name)
        print(kokku/len(self.data)*100,"%")
        print(sorted(luckyLoser.items(), key=lambda x: x[1], reverse=True))
        print()
                
    
    def showTabel(self):
        wins=[i for i in self.getTeamWins().values()]
        loses=[i for i in self.getTeamLoses().values()]
        draws=[i for i in self.getTeamDraws().values()]
        points=[i for i in self.getTeamPoints().values()]
        games=[i for i in self.getTeamGames().values()]
        tabel = {'Teams': list(self.teams),
                 'Wins': wins,
                 'Loses': loses,
                 'Draws': draws,
                 'Points': points,
                 'Games': games}
        df = pd.DataFrame(tabel)
        df=df.sort_values(by=['Points'],ascending=False)
        df=df.to_string(index=False)
        print(df)

In [76]:
season1718 = Football(season17_18,"Season 17_18")
season1819 = Football(season18_19,"Season 18_19")
season1920 = Football(season19_20,"Season 19_20")
#season1718.showTabel()
season1718.getWrongPrediction()
season1819.getWrongPrediction()
season1920.getWrongPrediction()

Season 17_18
18.684210526315788 %
[('Burnley', 8), ('Newcastle', 7), ('Watford', 6), ('West Ham', 5), ('Bournemouth', 5), ('Huddersfield', 5), ('Swansea', 4), ('Leicester', 4), ('Stoke', 4), ('Brighton', 4), ('Everton', 4), ('West Brom', 4), ('Crystal Palace', 4), ('Man United', 2), ('Man City', 1), ('Chelsea', 1), ('Tottenham', 1), ('Southampton', 1), ('Liverpool', 1), ('Arsenal', 0)]

Season 18_19
22.63157894736842 %
[('Burnley', 9), ('Cardiff', 8), ('West Ham', 7), ('Brighton', 7), ('Wolves', 7), ('Crystal Palace', 7), ('Leicester', 6), ('Southampton', 6), ('Newcastle', 5), ('Bournemouth', 5), ('Everton', 5), ('Watford', 5), ('Fulham', 2), ('Huddersfield', 2), ('Tottenham', 2), ('Arsenal', 1), ('Man United', 1), ('Chelsea', 1), ('Man City', 0), ('Liverpool', 0)]

Season 19_20
20.0 %
[('Bournemouth', 4), ('Brighton', 3), ('Newcastle', 3), ('West Ham', 2), ('Sheffield United', 2), ('Aston Villa', 2), ('Burnley', 2), ('Norwich', 2), ('Southampton', 2), ('Crystal Palace', 2), ('Leiceste